In [19]:
from PIL import Image
import numpy
import os
import re
from resizeimage import resizeimage

In [20]:
# resize images
# resizeImages('../data/yalefaces/', 'subject01')
def resizeImages(path, prefix):    
    for root, dirs, files in os.walk(path):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(path + name)
                resizedImg = resizeimage.resize_height(img, 50)
                resizedImg.save('../data/yalefaces_resized/'+name)

# images into array: http://stackoverflow.com/questions/13550376/pil-image-to-array-numpy-array-to-array-python
def readImagesIntoMatrix(path, prefix):
    images = None
    for root, dirs, files in os.walk(path):
        for name in files:
            if re.match(r'^'+prefix+'.*gif$', name, re.I):
                img = Image.open(path + name)
                (width, height) = img.size
                imgArr = numpy.asarray(img).reshape(height * width)/256
                if images is not None:
                    images = numpy.vstack([images, imgArr])
                else:
                    images = imgArr
    return images

In [22]:
imageList = readImagesIntoMatrix('yalefaces_resized/', 'subject.*happy\.')
print("Image list %s %s %s" % imageList.shape)

TypeError: not enough arguments for format string

In [14]:
(width, height) = 66, 50
# average Image
avgImg =  numpy.mean(imageList, axis=0)
Image.fromarray(numpy.uint8(avgImg.reshape(50,66))).save('test/avgImg.gif')

In [15]:
# Get EigenVectors
covMatrix = numpy.cov(imageList, rowvar=True)
eigenVal, eigenVec = numpy.linalg.eig(covMatrix)
# Make sure Eigenvectors are unit vectors
numpy.testing.assert_array_almost_equal(1.0, numpy.linalg.norm(eigenVec[0]))
Image.fromarray(numpy.uint8(largestEigenVec.reshape(50,66))).save('test/1stEigen.gif')

ValueError: total size of new array must be unchanged

In [9]:
# Reduce dimension thru eigen vector and save the resulting image
largestEigenVec = eigenVec[0]
print('Size of largest %s' % largestEigenVec.shape)

Size of largest 11
